In [1]:
import os
import re
import ast
import json
import spacy
import pickle
import difflib
import requests
import datetime
import jsonlines
import numpy as np
import pandas as pd

In [23]:
wn_val_pkl = "/work/pi_wenlongzhao_umass_edu/8/696-detecting-salient-entities/data/WN_pkl/WN_salience_ValSplit.pkl"
wn_val = "/work/pi_wenlongzhao_umass_edu/8/696-detecting-salient-entities/data/WN_csv/WN_salience_ValSplit.csv"

In [25]:
with open(wn_val_pkl, "rb") as f:
    wn_val_pkl = pickle.load(f)

/tmp/ipykernel_713689/3817052386.py:2: DeprecationWarning: numpy.core.numeric is deprecated and has been renamed to numpy._core.numeric. The numpy._core namespace contains private NumPy internals and its use is discouraged, as NumPy internals can change without warning in any release. In practice, most real-world usage of numpy.core is to access functionality in the public NumPy API. If that is the case, use the public NumPy API. If not, you are using NumPy internals. If you would still like to access an internal attribute, use numpy._core.numeric._frombuffer.
  wn_val_pkl = pickle.load(f)


In [26]:
wn_val_pkl.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15644 entries, 0 to 15643
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   text             15644 non-null  object        
 1   date             15644 non-null  datetime64[ns]
 2   title            15644 non-null  object        
 3   entity title     15603 non-null  object        
 4   entity salience  15644 non-null  int64         
 5   offsets          15644 non-null  object        
 6   url              15544 non-null  object        
dtypes: datetime64[ns](1), int64(1), object(5)
memory usage: 855.7+ KB


In [29]:
wn_val_pkl[wn_val_pkl['entity title'].isna()==True].info()

<class 'pandas.core.frame.DataFrame'>
Index: 41 entries, 193 to 14029
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   text             41 non-null     object        
 1   date             41 non-null     datetime64[ns]
 2   title            41 non-null     object        
 3   entity title     0 non-null      object        
 4   entity salience  41 non-null     int64         
 5   offsets          41 non-null     object        
 6   url              1 non-null      object        
dtypes: datetime64[ns](1), int64(1), object(5)
memory usage: 2.6+ KB


In [32]:
wn_val = pd.read_csv(wn_val)

In [14]:
column_mapping = {
    'text': 'article_text',
    'title': 'article_title',
    'entity title': 'entity_title',
    'entity salience': 'entity_salience'
}
wn_val.rename(columns=column_mapping, inplace=True)

In [33]:
wn_val.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15524 entries, 0 to 15523
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   text             15524 non-null  object
 1   date             15524 non-null  object
 2   title            15524 non-null  object
 3   entity title     15523 non-null  object
 4   entity salience  15524 non-null  int64 
 5   offsets          15524 non-null  object
 6   wiki_ID          15524 non-null  int64 
dtypes: int64(2), object(5)
memory usage: 849.1+ KB


In [ ]:
wn_val.info()

In [37]:
wn_val[wn_val['entity title'].isna()==True]
# wn_val['entity_title'].unique()
# wn_val[wn_val['wiki_ID'].isna()==True]

,text,date,title,entity title,entity salience,offsets,wiki_ID
13909,On Sunday in tennis's French Open men's single...,2017-06-13,¡La décima! Nadal again a French CHAMP10N,NaN,0,"(277, 277)",-1


In [ ]:
wn_val.info()

In [ ]:
wn_val = wn_val.dropna(subset=['entity_title'])

In [1]:
from rdflib import Graph

g = Graph()
g.parse("reuters-128-entity-salience.ttl", format="turtle")

for subj, pred, obj in g:
    print(f"Subject: {subj}, Predicate: {pred}, Object: {obj}")

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



Subject: http://aksw.org/N3/Reuters-128/66#char=382,392, Predicate: http://persistence.uni-leipzig.org/nlp2rdf/ontologies/nif-core#referenceContext, Object: http://aksw.org/N3/Reuters-128/66#char=0,1268
Subject: http://aksw.org/N3/Reuters-128/118#char=1877,1883, Predicate: http://persistence.uni-leipzig.org/nlp2rdf/ontologies/nif-core#beginIndex, Object: 1877
Subject: http://aksw.org/N3/Reuters-128/20#char=0,20, Predicate: http://www.w3.org/1999/02/22-rdf-syntax-ns#type, Object: http://persistence.uni-leipzig.org/nlp2rdf/ontologies/nif-core#NamedEntity
Subject: http://aksw.org/N3/Reuters-128/31#char=346,351, Predicate: http://www.w3.org/1999/02/22-rdf-syntax-ns#type, Object: http://persistence.uni-leipzig.org/nlp2rdf/ontologies/nif-core#RFC5147String
Subject: http://aksw.org/N3/Reuters-128/105#char=1635,1648, Predicate: http://persistence.uni-leipzig.org/nlp2rdf/ontologies/nif-core#endIndex, Object: 1648
Subject: http://aksw.org/N3/Reuters-128/86#char=495,502, Predicate: http://persist

In [3]:
from rdflib.namespace import RDF
from rdflib import URIRef

nif_ns = "http://persistence.uni-leipzig.org/nlp2rdf/ontologies/nif-core#"

for s, p, o in g:
    if str(p) == RDF.type:
        print(f"{s} is of type {o}")
    elif nif_ns + "beginIndex" in str(p):
        print(f"{s} starts at character index {o}")
    elif nif_ns + "endIndex" in str(p):
        print(f"{s} ends at character index {o}")
    elif nif_ns + "referenceContext" in str(p):
        print(f"{s} refers to context {o}")

http://aksw.org/N3/Reuters-128/94#char=772,780 ends at character index 780
http://aksw.org/N3/Reuters-128/58#char=830,842 refers to context http://aksw.org/N3/Reuters-128/58#char=0,870
http://aksw.org/N3/Reuters-128/33#char=1825,1830 ends at character index 1830
http://aksw.org/N3/Reuters-128/35#char=244,253 starts at character index 244
http://aksw.org/N3/Reuters-128/117#char=551,587 ends at character index 587
http://aksw.org/N3/Reuters-128/99#char=61,78 refers to context http://aksw.org/N3/Reuters-128/99#char=0,517
http://aksw.org/N3/Reuters-128/76#char=379,414 refers to context http://aksw.org/N3/Reuters-128/76#char=0,764
http://aksw.org/N3/Reuters-128/68#char=241,247 ends at character index 247
http://aksw.org/N3/Reuters-128/51#char=132,135 refers to context http://aksw.org/N3/Reuters-128/51#char=0,578
http://aksw.org/N3/Reuters-128/66#char=1250,1255 ends at character index 1255
http://aksw.org/N3/Reuters-128/74#char=93,98 ends at character index 98
http://aksw.org/N3/Reuters-128/

In [5]:
df = pd.read_csv("/work/pi_wenlongzhao_umass_edu/8/696-detecting-salient-entities/data/WN-salience-test-NER-output.csv")

In [6]:
df.head()

,title,text,entities
0,One dead as Super Typhoon Megi makes landfall ...,One man is confirmed drowned in a river as Sup...,"[{""mention"": ""One"", ""begin_offset"": 0, ""end_of..."
1,"'Enough, enough, enough, enough': 1700 women m...","Over 1,700 women from more than 40 nations mar...","[{""mention"": ""Over 1,700"", ""begin_offset"": 0, ..."
2,Australian Dollar reaches a 27 year high,"After 27 years, the Australian Dollar has achi...","[{""mention"": ""27 years"", ""begin_offset"": 6, ""e..."
3,"Two Santa Clara, Utah teens in critical condit...","Two Santa Clara, Utah teenagers were criticall...","[{""mention"": ""Two"", ""begin_offset"": 0, ""end_of..."
4,Charlie the smoking chimpanzee dies aged 52,"Charlie, a Chimpanzee in a South African zoo w...","[{""mention"": ""Charlie"", ""begin_offset"": 0, ""en..."


In [7]:
df.iloc[0].entities

'[{"mention": "One", "begin_offset": 0, "end_offset": 3}, {"mention": "Super Typhoon Megi", "begin_offset": 43, "end_offset": 61}, {"mention": "Juan", "begin_offset": 77, "end_offset": 81}, {"mention": "Philippines", "begin_offset": 105, "end_offset": 116}, {"mention": "Megi", "begin_offset": 118, "end_offset": 122}, {"mention": "this year", "begin_offset": 170, "end_offset": 179}, {"mention": "Manila", "begin_offset": 233, "end_offset": 239}, {"mention": "one", "begin_offset": 259, "end_offset": 262}, {"mention": "60 km/h", "begin_offset": 298, "end_offset": 305}, {"mention": "35 mph", "begin_offset": 307, "end_offset": 313}, {"mention": "Megi", "begin_offset": 328, "end_offset": 332}, {"mention": "190 km/h", "begin_offset": 386, "end_offset": 394}, {"mention": "120 mph", "begin_offset": 396, "end_offset": 403}, {"mention": "Megi", "begin_offset": 427, "end_offset": 431}, {"mention": "up to 60%", "begin_offset": 445, "end_offset": 454}, {"mention": "Cagayan", "begin_offset": 479, "end